In [48]:
#!/cluster/bh0085/anaconda27/envs/py3/bin/python
##
# IMPORTS BOILERPLATE
##
import pandas as pd
import numpy as np
import itertools as it
import os,re, sys
from collections import defaultdict

sys.path.append("/cluster/bh0085")
from mybio import util
from _config import DATA_DIR, OUT_PLACE, N_SPLITS, QSUBS_DIR, EXP_NAMES, OLIGO_LIBRARY, POSITIVE_CONTROLS_FILE
import _config

#IO DIRECTORY CONFIG
try:NAME = util.get_fn(__file__)
except NameError:NAME="0626_c1a_dictionary_reports"

OUT_DIR = os.path.join(OUT_PLACE, NAME)
util.ensure_dir_exists(OUT_DIR)

##
# CUSTOM CODE
##


#load oligo library from experimental design
oligos_lib = OLIGO_LIBRARY
oligos_lib["id"] = oligos_lib.index
positive_controls = pd.read_csv(POSITIVE_CONTROLS_FILE)


from _config import DATA_DIR, OUT_PLACE, N_SPLITS, QSUBS_DIR, EXP_NAMES
TX_INP_DIR = os.path.join(OUT_PLACE, "c0_bin_transcript_umis")
OLIGO_INP_DIR = os.path.join(OUT_PLACE, "b1_demultiplex_oligos")


#load oligo library from experimental design
oligos_lib = OLIGO_LIBRARY
oligos_lib["id"] = oligos_lib.index
positive_controls = pd.read_csv(POSITIVE_CONTROLS_FILE)




bc_substr_ids = dict([((s1+s2+s3) ,sum([     {"A":0,"T":1,"C":2,"G":3}[(s1+s2+s3)[i]]*(4**i) for i in range(3)])) for s1 in "ATGC" for s2 in "ATGC" for s3 in "ATGC"])
                     
#bc_id_substrs = dict([[(s1+s2+s3) ,sum([ {"A":0,"T":1,"C":2,"G":3}[(s1+s2+s3)[i]*(4**i)] for i in range(3)])] for s1 in "ATGC" for s2 in "ATGC" for s3 in "ATGC"])
bc_substr_inc = 4
bc_substr_max = len(bc_substr_ids)


In [49]:
split="0"
bc_substr_idx=0

In [94]:
all_oligos = pd.DataFrame()
all_tx_bcs = set()
all_oligo_bcs = set()

bc_substr_idx = int(bc_substr_idx)

merged_results = pd.DataFrame()
tx_discovery_rates = []
oligo_mapping_multiplicity = []
total_txs = 0
total_oligos = 0
#load barcode-matched umis from dictionary reads

cnt = -1
for oligo_file in os.listdir(OLIGO_INP_DIR):
    #file_format = "../out/b1_demultiplex_oligos/20190320-100XSTARRseq-Nextera-PE_HKNM2AFXY_S1_L001_3.json"
    if not ("_"+split+".json" in oligo_file):
        continue
    if "SHE3202-STAR" in oligo_file: continue



    a_output2 = pd.read_csv(os.path.join(OLIGO_INP_DIR,oligo_file))
    a_output2 = a_output2.rename({"umi":"bc"}, axis = "columns")
    a_output_15 = a_output2.loc[a_output2.bc.str.len() == 15]
    
    all_oligos.append(a_output_15.assign(fname=oligo_file))
    
    total_oligos+= len(a_output_15)
    all_oligo_bcs.update(list(a_output_15.bc))

    

    for k, ogroup in a_output_15.groupby(a_output_15.bc.str.slice(0,3)):
        print(k)
        if "N" in k: 
          continue

        if (bc_substr_ids[k] < bc_substr_idx ) or ( bc_substr_ids[k] >= (bc_substr_idx + bc_substr_inc) ) :
          continue 


        for exp in EXP_NAMES:
            cnt+=1


            
            tx_file = os.path.join(TX_INP_DIR,"{}_{}.csv".format(exp,k))
            
            
            print(f"oligo file: {oligo_file}\ntx_file: {tx_file}")
            tx_subdict_df = pd.read_csv(tx_file)[["bc","umi"]]
            tx_subdict_df["exp"] = exp
            total_txs += len(tx_subdict_df)
            joined = pd.merge(ogroup.rename({"count":"oligo_reads"},axis="columns"),tx_subdict_df.groupby(["bc","umi","exp"]).size().rename("dictionary_reads").reset_index())
            merged_results = merged_results.append(joined)
            print(cnt)
            oligo_mapping_multiplicity.append(float(len(joined)) / len(ogroup))
            tx_discovery_rates.append(float(len(joined)) / len(tx_subdict_df))
            
        break
    break
        
        
merged_results.to_csv(os.path.join(OUT_DIR,f"merged_{bc_substr_idx}_{split}.csv"),index=False)
import json
print(oligo_mapping_success_rates)
print(tx_discovery_rates)
print(f"""<json>{json.dumps({
    "oligo_mapping_success_rates":oligo_mapping_multiplicity,
    "tx_discovery_rates":tx_discovery_rates
})}</json>""")


AAA
oligo file: 20190320-100XSTARRseq-Nextera-PE_HKNM2AFXY_S1_L004_0.json
tx_file: /cluster/bh0085/prj/starr_all/starr/out/c0_bin_transcript_umis/U2OS_WT_BR1_AAA.csv
0
oligo file: 20190320-100XSTARRseq-Nextera-PE_HKNM2AFXY_S1_L004_0.json
tx_file: /cluster/bh0085/prj/starr_all/starr/out/c0_bin_transcript_umis/U2OS_WT_BR2_AAA.csv
1
oligo file: 20190320-100XSTARRseq-Nextera-PE_HKNM2AFXY_S1_L004_0.json
tx_file: /cluster/bh0085/prj/starr_all/starr/out/c0_bin_transcript_umis/HCT116_WT_BR1_AAA.csv
2
oligo file: 20190320-100XSTARRseq-Nextera-PE_HKNM2AFXY_S1_L004_0.json
tx_file: /cluster/bh0085/prj/starr_all/starr/out/c0_bin_transcript_umis/HCT116_WT_BR2_AAA.csv
3
oligo file: 20190320-100XSTARRseq-Nextera-PE_HKNM2AFXY_S1_L004_0.json
tx_file: /cluster/bh0085/prj/starr_all/starr/out/c0_bin_transcript_umis/HCT116_Gemcitabine2uM_BR1_AAA.csv
4
oligo file: 20190320-100XSTARRseq-Nextera-PE_HKNM2AFXY_S1_L004_0.json
tx_file: /cluster/bh0085/prj/starr_all/starr/out/c0_bin_transcript_umis/HCT116_Gemcitabi

In [95]:
len(merged_results)

149912

In [97]:
merged_results.groupby("exp").size()

exp
DLD1_WT_BR1                                        13125
DLD1_WT_BR2                                        13002
HCT116_Gemcitabine2uM_BR1                           2686
HCT116_Gemcitabine2uM_BR2                           3113
HCT116_WT_BR1                                       8431
HCT116_WT_BR2                                       7359
U2OS_NFKB1KO_Sorted_GFPneg_BR1_with_ML120B_20uM     5192
U2OS_NFKB1KO_Sorted_GFPneg_BR2_with_ML120B_20uM     5527
U2OS_NFKB1KO_Sorted_GFPpos_BR1                      4564
U2OS_NFKB1KO_Sorted_GFPpos_BR2                      4801
U2OS_NFKB2KO_Sorted_GFPneg_BR1_with_ML120B_20uM     6254
U2OS_NFKB2KO_Sorted_GFPneg_BR2_with_ML120B_20uM     4729
U2OS_NFKB2KO_Sorted_GFPpos_BR1                      9598
U2OS_NFKB2KO_Sorted_GFPpos_BR2                      8243
U2OS_WT_BR1                                        14456
U2OS_WT_BR2                                        12541
other                                              26291
dtype: int64

In [96]:
len(merged_results.loc[merged_results.exp=="other"])

26291

In [83]:
len(merged_results)

149912

In [85]:
merged_results.umi.nunique()

115541

In [74]:
merged = pd.merge(ogroup.rename({"count":"oligo_reads"},axis="columns"),tx_subdict_df.groupby(["bc","umi"]).size().rename("dictionary_reads").reset_index())

In [75]:
len(merged)

26291

In [76]:
len(merged_results)

938278

In [90]:
len(merged_results.loc[merged_results.umi.str.contains("N")])

181

In [58]:
merged_results.umi.nunique()

115541

In [ ]:
os.path.join(OUT_DIR,f"merged_{bc_substr_idx}_{split}.csv")

In [ ]:
48 * 64

In [ ]:
len(merged_results)

In [ ]:
len(tx_subdict_df.umi.unique())

In [ ]:
unique_bcs = set(tx_subdict_df.bc)

In [ ]:
unique_oligo_bcs = set(ogroup.bc)

In [ ]:
len(unique_oligo_bcs) * 64

In [ ]:
len(unique_bcs)

In [ ]:
len(unique_oligo_bcs)

In [ ]:
4608*64

In [ ]:
len(unique_bcs.intersection(unique_oligo_bcs))

In [ ]:
print(total_txs)

In [ ]:
print(total_oligos)

In [ ]:
len(all_oligo_bcs)

In [ ]:
len(all_tx_bcs)

In [ ]:
len(merged_results)

In [ ]:
len(tx_subdict_df.loc[tx_subdict_df.bc.str.slice(0,3)=="AAA"])

In [ ]:
len(ogroup)

In [ ]:
len(ogroup.loc[ogroup.bc.str.slice(0,3)=="AAA"])

In [ ]:
len(merged_results)

In [ ]:
len(merged_results)

In [ ]:
len(ogroup)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ogroup.bc.nunique()

In [ ]:
oligo_mapping_success_rates

In [ ]:
plt.plot(oligo_mapping_success_rates)

In [ ]:
merged_results.bc.nunique()

In [ ]:
len(merged_results)

In [ ]:
len(all_tx_bcs)

In [ ]:
tx_subdict_df.bc.nunique()

In [ ]:
ogroup.bc.nunique()

In [ ]:
merged_results.umi.nunique()

In [ ]:
merged_results.bc.nunique()

In [ ]:
len(merged_results)

In [ ]:
len(os.listdir(OLIGO_INP_DIR))

In [ ]:
len(ogroup)

In [ ]:
len(ogroup.bc.unique())

In [ ]:
len(EXP_NAMES)

In [ ]:
len(oligo_mapping_success_rates)

In [ ]:
tx_discovery_rates

In [ ]:
oligo_mapping_success_rates